# ✔ CUDA setup

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!nvidia-smi

Tue Feb  8 19:00:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-js74dqug
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-js74dqug
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=16b6961167beab8338d7fbf0d8487590db137d6b744b445ef50bad37941532c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-_m9fu__m/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

directory /content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel/src already exists
Out bin /content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel/result.out


# ✔ VS Code on Colab

In [ ]:
# 1. Install the colab-code package...
!pip install colabcode

In [ ]:
# 2. Import and launch...
from colabcode import ColabCode
ColabCode()

Code Server can be accessed on: NgrokTunnel: "https://1636-35-185-135-22.ngrok.io" -> "http://localhost:10000"
[2021-10-16T17:00:56.429Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2021-10-16T17:00:56.431Z] info  Using user-data-dir ~/.local/share/code-server
[2021-10-16T17:00:56.450Z] info  Using config file ~/.config/code-server/config.yaml
[2021-10-16T17:00:56.450Z] info  HTTP server listening on http://127.0.0.1:10000 
[2021-10-16T17:00:56.450Z] info    - Authentication is disabled 
[2021-10-16T17:00:56.450Z] info    - Not serving HTTPS 
 INFO Installing extension: ms-vscode.cpptools
 INFO Downloaded extension: ms-vscode.cpptools /root/.local/share/code-server/CachedExtensionVSIXs/ms-vscode.cpptools-1.5.1
 INFO Extracted extension to /root/.local/share/code-server/extensions/.801240b6-e04b-4087-9084-3b6369662ed5: ms-vscode.cpptools
 INFO Renamed to /root/.local/share/code-server/extensions/ms-vscode.cpptools-1.5.1
 INFO Installation completed. ms-vscode.cppt

#DeviceQuery

In [ ]:
#@title working directory: **deviceQuery/**
%cd /home/grossi/CUDA/lab4/deviceQuery/
%ls

In [ ]:
%%writefile /home/grossi/CUDA/lab4/deviceQuery/helper.h
// Beginning of GPU Architecture definitions
inline int _ConvertSMVer2Cores(int major, int minor) {
	// Defines for GPU Architecture types (using the SM version to determine
	// the # of cores per SM
	typedef struct {
		int SM;  // 0xMm (hexidecimal notation), M = SM Major version,
		// and m = SM minor version
		int Cores;
	} sSMtoCores;

	sSMtoCores nGpuArchCoresPerSM[] = {
			{0x20, 32},
			{0x30, 192},
			{0x32, 192},
			{0x35, 192},
			{0x37, 192},
			{0x50, 128},
			{0x52, 128},
			{0x53, 128},
			{0x60,  64},
			{0x61, 128},
			{0x62, 128},
			{0x70,  64},
			{0x72,  64},
			{0x75,  64},
			{-1, -1}};

	int index = 0;

	while (nGpuArchCoresPerSM[index].SM != -1) {
		if (nGpuArchCoresPerSM[index].SM == ((major << 4) + minor)) {
			return nGpuArchCoresPerSM[index].Cores;
		}

		index++;
	}

	// If we don't find the values, we default use the previous one
	// to run properly
	printf(
			"MapSMtoCores for SM %d.%d is undefined."
			"  Default to use %d Cores/SM\n",
			major, minor, nGpuArchCoresPerSM[index - 1].Cores);
	return nGpuArchCoresPerSM[index - 1].Cores;
}


In [ ]:
%%writefile /home/grossi/CUDA/lab4/deviceQuery/deviceQuery.cu
#include <stdlib.h>
#include <stdio.h>
#include "helper.h"
#include "../../utils/common.h"

int main(void) {

	printf("\nCUDA Device Query (Runtime API) version (CUDART static linking)\n\n");
	int deviceCount = 0;
	CHECK(cudaGetDeviceCount(&deviceCount));

	// This function call returns 0 if there are no CUDA capable devices.
	if (deviceCount == 0)
		printf("There are no available device(s) that support CUDA\n");
	else
		printf("Detected %d CUDA Capable device(s)\n", deviceCount);

	int dev, driverVersion = 0, runtimeVersion = 0;

	for (dev = 0; dev < deviceCount; ++dev) {
		cudaSetDevice(dev);
		cudaDeviceProp deviceProp;
		cudaGetDeviceProperties(&deviceProp, dev);

		printf("\nDevice %d: \"%s\"\n", dev, deviceProp.name);

		cudaDriverGetVersion(&driverVersion);
		cudaRuntimeGetVersion(&runtimeVersion);

		printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
				driverVersion / 1000, (driverVersion % 100) / 10,
				runtimeVersion / 1000, (runtimeVersion % 100) / 10);

		printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
				deviceProp.major, deviceProp.minor);

		printf("  Total amount of global memory:                 %.0f MBytes (%llu bytes)\n",
				(float) deviceProp.totalGlobalMem / 1048576.0f,
				(unsigned long long) deviceProp.totalGlobalMem);

	    printf("  (%2d) Multiprocessors, (%3d) CUDA Cores/MP:     %d CUDA Cores\n",
	           deviceProp.multiProcessorCount,
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor),
	           _ConvertSMVer2Cores(deviceProp.major, deviceProp.minor) *
	               deviceProp.multiProcessorCount);

		printf("  GPU Max Clock rate:                            %.0f MHz (%0.2f GHz)\n",
				deviceProp.clockRate * 1e-3f, deviceProp.clockRate * 1e-6f);

		printf("  Memory Clock rate:                             %.0f Mhz\n", deviceProp.memoryClockRate * 1e-3f);
		printf("  Memory Bus Width:                              %d-bit\n", deviceProp.memoryBusWidth);
		if (deviceProp.l2CacheSize)
			printf("  L2 Cache Size:                                 %d bytes\n", deviceProp.l2CacheSize);

		printf("  Maximum Texture Dimension Size (x,y,z)         1D=(%d), 2D=(%d, %d), 3D=(%d, %d, %d)\n",
				deviceProp.maxTexture1D, deviceProp.maxTexture2D[0],
				deviceProp.maxTexture2D[1], deviceProp.maxTexture3D[0],
				deviceProp.maxTexture3D[1], deviceProp.maxTexture3D[2]);

		printf("  Maximum Layered 1D Texture Size, (num) layers  1D=(%d), %d layers\n",
				deviceProp.maxTexture1DLayered[0],
				deviceProp.maxTexture1DLayered[1]);

		printf("  Maximum Layered 2D Texture Size, (num) layers  2D=(%d, %d), %d layers\n",
				deviceProp.maxTexture2DLayered[0],
				deviceProp.maxTexture2DLayered[1],
				deviceProp.maxTexture2DLayered[2]);

		printf("  Total amount of constant memory:               %lu bytes\n",
				deviceProp.totalConstMem);
		printf("  Total amount of shared memory per block:       %lu bytes\n",
				deviceProp.sharedMemPerBlock);
		printf("  Total number of registers available per block: %d\n",
				deviceProp.regsPerBlock);
		printf("  Warp size:                                     %d\n",
				deviceProp.warpSize);
		printf("  Maximum number of threads per multiprocessor:  %d\n",
				deviceProp.maxThreadsPerMultiProcessor);
		printf("  Maximum number of threads per block:           %d\n",
				deviceProp.maxThreadsPerBlock);
		printf("  Max dimension size of a thread block (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxThreadsDim[0], deviceProp.maxThreadsDim[1],
				deviceProp.maxThreadsDim[2]);
		printf("  Max dimension size of a grid size    (x,y,z): (%d, %d, %d)\n",
				deviceProp.maxGridSize[0], deviceProp.maxGridSize[1],
				deviceProp.maxGridSize[2]);
		printf("  Maximum memory pitch:                          %lu bytes\n",
				deviceProp.memPitch);
		printf("  Texture alignment:                             %lu bytes\n",
				deviceProp.textureAlignment);
		printf("  Concurrent copy and kernel execution:          %s with %d copy engine(s)\n",
				(deviceProp.deviceOverlap ? "Yes" : "No"),
				deviceProp.asyncEngineCount);
		printf("  Run time limit on kernels:                     %s\n",
				deviceProp.kernelExecTimeoutEnabled ? "Yes" : "No");
		printf("  Integrated GPU sharing Host Memory:            %s\n",
				deviceProp.integrated ? "Yes" : "No");
		printf("  Support host page-locked memory mapping:       %s\n",
				deviceProp.canMapHostMemory ? "Yes" : "No");
		printf("  Alignment requirement for Surfaces:            %s\n",
				deviceProp.surfaceAlignment ? "Yes" : "No");
		printf("  Device has ECC support:                        %s\n",
				deviceProp.ECCEnabled ? "Enabled" : "Disabled");

		printf("  Device supports Unified Addressing (UVA):      %s\n",
				deviceProp.unifiedAddressing ? "Yes" : "No");
		printf("  Device PCI Domain ID / Bus ID / location ID:   %d / %d / %d\n",
				deviceProp.pciDomainID, deviceProp.pciBusID,
				deviceProp.pciDeviceID);
	}
	return 0;
}


In [ ]:
# Compilazione ed esecuzione

!nvcc deviceQuery.cu -o deviceQuery
!deviceQuery

In [ ]:
# Occupancy
!nvprof --metrics achieved_occupancy ./FlockingBehaviourPar

==2745== NVPROF is profiling process 2745, command: ./FlockingBehaviourPar
device 0: Tesla K80
==2745== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.

threadsNum: 4000
effthreadsNum: 4096
genperthread: 2500


GPU Flock generation...
    GPU elapsed time: 0.03840 (sec)
Boid 0: pos(22.2, 44.03, 38.63); dir(-0.5449, 0.6648, -0.511)
Boid 9999999: pos(6.172, 11.05, -38.18); dir(-0.1124, -0.08378, -0.9901)

threadsNum: 10000000
effthreadsNum: 10000000


GPU Neighborhoods computation...
    GPU elapsed time: 0.00000 (sec)
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
first neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
last neigh: 0
==2745== Profiling application: ./FlockingBehaviourPar
==2745== Profiling result:
==2745== Metric

In [ ]:
# Spilling
!nvcc --ptxas-options=-v -arch=sm_37 FlockingBehaviourPar.cu

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
ptxas info    : 64448 bytes gmem, 72 bytes cmem[3]
ptxas info    : Compiling entry function '_Z19generateBoidsStatusijijjjPfP17curandStateXORWOW' for 'sm_37'
ptxas info    : Function properties for _Z19generateBoidsStatusijijjjPfP17curandStateXORWOW
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 49 registers, 360 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compiling entry function '_Z16initializeStatesjP17curandStateXORWOW' for 'sm_37'
ptxas info    : Function properties for _Z16initializeStatesjP17curandStateXORWOW
    6440 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 40 registers, 336 bytes cmem[0]
/tmp/tmpxft_0000064e_00000000-11_FlockingBehaviourPar.o: In function `main':
tmpxft_0000064e_000

# Flocking Behaviour Sequential

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq     FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [ ]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourParallel'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [ ]:
%cd ..
%cd "FlockingBehaviourSequential"
%ls

/content/drive/MyDrive/GPU Computing/Progetto
/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourSequential
Boid.cpp  Flock.cpp  FlockingBehaviourSeq     FlockingBehaviourSeq.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourSeq.cu  src/                    Utils.h


In [ ]:
%%writefile FlockingBehaviourSeq.h
#include "Boid.h"
#include "Flock.h"
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <map> 
#include <vector> 

typedef struct BoidData {
    float p1;
    float p2;
    float p3;
    float d1;
    float d2;
    float d3;
} BoidData;

void computeNeighborhoods();

void updateFlock(float);
void getSeparationDirection(int, float*);
void getCohesionDirection(int, float*);
void getAlignDirection(int, float*);
void moveBoid(int, float);

void printNeighborhoods();
void printFlock();

Overwriting FlockingBehaviourSeq.h


In [ ]:
%%writefile FlockingBehaviourSeq.cu
#include "FlockingBehaviourSeq.h"

float velocity = 20; // boid velocity in meters per second
double updateTime = 1.5; // update time of the simulation in seconds
float separationWeigth = 1; // weight of the separation component in the blending
float cohesionWeigth = 1; // weight of the cohesion component in the blending
float alignWeigth = 1; // weight of the align component in the blending
int flockDim = 10000; // 10000000 number of boids in the flock
float neighDim = 200; // 75000000 dimension of the neighborhood in meters
int minRand = -50000; // -50000 minimum value that can be generated for initial position and direction
int maxRand = 50000; // 50000 maximum value that can be generated for initial position and direction
float decimals = 3; // 3 number of decimal digits in the generated values for initial position and direction
int iterations = 1; // number of updates 

float* flockData;
bool* neighborhoods;

int main(void) {

	srand (time(NULL));
	float div = pow(10.0, decimals);
	int numsToGenerate = flockDim * 6;

	// generate boids with random initial position and direction and add them to the flock
	printf("\n\nCPU Flock generation...\n");
	double cpuTimeStart = seconds();

	flockData = (float*) malloc(numsToGenerate * sizeof(float));
	for(int i = 0; i < numsToGenerate; i+=6){
			flockData[i] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+1] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+2] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+3] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+4] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockData[i+5] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
		
			vector3Normalize(flockData+i+3);
	}

	double cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	for(int i = 0; i < 12; i++){
			printf("first: %.5f\n", flockData[i]);
	}
	for(int i = 0; i < 12; i++){
			printf("last: %.5f\n", flockData[numsToGenerate - i - 1]);
	}

	// calculate neighborhoods (data stored in a boolean matrix)
  neighborhoods = (bool*) malloc(flockDim * flockDim * sizeof(bool));

	printf("\n\nCPU neighborhoods generation...\n");
	cpuTimeStart = seconds();
	computeNeighborhoods();
	cpuTime = seconds() - cpuTimeStart;
	printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

	//printFlock();		
	//std::cout << std::endl;
	//printNeighborhoods();

	// start simulation loop that updates the flock each updateTime
	double loopStart = seconds();
	double tmpTime = updateTime;
	while(iterations > 0){

	 		auto duration = seconds() - loopStart;
	 		if(duration >= tmpTime)
			{
				 	printf("\n\nCPU Flock update...\n");
					cpuTimeStart = seconds();
					updateFlock(updateTime);
					cpuTime = seconds() - cpuTimeStart;
					printf("    CPU elapsed time: %.5f (sec)", cpuTime);
				
					tmpTime += updateTime;
					iterations--;

					//std::cout << std::endl;
				  //printFlock();
				
					//std::cout << std::endl;
					//printNeighborhoods();
			}
	}

	free(flockData);
	free(neighborhoods);
	
	return 0;
}

void computeNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				for(int j = 0; j < flockDim; j++){
						if(j > i){	
								neighborhoods[i*flockDim+j] = vector3Distance(flockData+i*6, flockData+j*6) <= neighDim;
						}
						else if(i == j){
								neighborhoods[i*flockDim+j] = 0;
						}
						else{
								neighborhoods[i*flockDim+j] = neighborhoods[j*flockDim+i];
						}
				}
    }
}

void printFlock(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i*6] << ", " << flockData[i*6+1] << ", " << flockData[i*6+2] << 
    		"); dir(" << flockData[i*6+3] << ", " << flockData[i*6+4] << ", " << flockData[i*6+5] << ")" << std::endl;
    }
}

void printNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoods[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }
}

void updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(int i = 0; i < flockDim; i++){
      
        getSeparationDirection(i, separation);
        getCohesionDirection(i, cohesion);
        getAlignDirection(i, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            flockData[i*6+3] = finalDirection[0];
						flockData[i*6+4] = finalDirection[1];
						flockData[i*6+5] = finalDirection[2];
        }

        moveBoid(i, time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    computeNeighborhoods();
}

void getSeparationDirection(int i, float* separation){
    
    float* tmp = (float*) malloc(3*sizeof(float));
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
						vector3Sub(flockData+i*6, flockData+j*6, tmp);
						vector3Normalize(tmp);
						vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001), tmp);
						vector3Sum(separation, tmp, separation);
				}
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void getCohesionDirection(int i, float* cohesion){
    
    float count = 0.0;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
						vector3Sum(cohesion, flockData+j*6, cohesion);
						count++;
				}
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, flockData+i*6, cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void getAlignDirection(int i, float* align){
    
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
        		vector3Sum(align, flockData+j*6+3, align);
				}
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void moveBoid(int i, float time) { 
    
    for(int j = 0; j < 3; j++){
        flockData[i*6+j] += flockData[i*6+3+j] * velocity * time;
    }
}

Overwriting FlockingBehaviourSeq.cu


In [ ]:
%%writefile Boid.h
#include <vector>
#include <iostream>
#include <iomanip>
#include <string>

#ifndef BOID_H
#define BOID_H
class Boid{
    
    public:
    Boid();

    Boid(const int, const float, float*, float*);

    float* getDirection() const;
    float* getPosition() const;
    const int& getId() const;
    void setDirection(float*);
    void setPosition(float*);

    // move the boid by updating its position based on its current direction, velocity and time passed since the last update
    void move(float);

    void print() const;

    private:
    int id; // unique id of the boid
    float velocity; // boid velocity in meters per second
    float* position; // one unit is one meter
    float* direction; // normalized vector
};
#endif

Overwriting Boid.h


In [ ]:
%%writefile Boid.cpp
#include "Boid.h"

Boid::Boid(const int id, const float vel, float* pos, float* dir): id{id}, velocity{vel}, direction{dir}, position{pos} {}

float* Boid::getDirection() const { return direction; }

float* Boid::getPosition() const { return position; }

const int& Boid::getId() const { return id; }

void Boid::setDirection(float* dir) { 
    direction[0] = dir[0]; 
    direction[1] = dir[1]; 
    direction[2] = dir[2]; 
}

void Boid::setPosition(float* pos) { 
    position[0] = pos[0];
    position[1] = pos[1];
    position[2] = pos[2];
}

void Boid::move(float time) { 
    
    for(int i = 0; i < 3; i++){
        position[i] += direction[i] * velocity * time;
    }
}

void Boid::print() const{
    std::cout << std::setprecision(4) << "Boid " << id << ": " << "pos(" << position[0] << ", " << position[1] << ", " << position[2] << 
    "); dir(" << direction[0] << ", " << direction[1] << ", " << direction[2] << ")" << std::endl;
}

Overwriting Boid.cpp


In [ ]:
%%writefile Flock.h
#include <iostream>
#include <map>
#include <vector>
#include "Boid.h"
#include "Utils.h"

#ifndef FLOCK_H
#define FLOCK_H
class Flock{
    
    public:
    Flock();
    Flock(std::map<int, Boid>&&);

    const std::map<int, std::vector<int>>& getNeighborhoodMap() const;
    const std::map<int, Boid>& getBoidsMap() const;

    const std::vector<int>& getNeighborhood(const Boid&) const;
    void setNeighborhoodDim(const int);
    void setBlendingWeigths(const int, const int, const int);

    // returns the list of boids that are nearer than neighborhoodDim from the passed boid
    std::vector<int> computeNeighborhood(const Boid&);

    // adds a boid to the flock
    void addBoid(const Boid&);

    // computes the neighborhoods of each boid based on the current situation and puts them inside the neighborhoodMap
    void updateNeighborhoodMap();

    // for each boid computes all the components that influence its direction, blends them and set the resulting 
    // direction as the current direction of the boid only if it is not (0,0,0). otherwise the old direction is maintained
    void updateFlock(float);

    void getSeparationDirection(const int, float*) const;
    void getCohesionDirection(const int, float*) const;
    void getAlignDirection(const int, float*) const;

    void print() const;
    void printNeighborhoods() const;

    private:
    std::map<int, std::vector<int>> neighborhoodMap; // associates the neighborhood of a boid to the id of the boid
    std::map<int, Boid> boidsMap; // associates a boid to its id
    float neighborhoodDim; // dimension of the neighborhood in meters
    float separationWeigth; // weight of the separation component in the blending
    float cohesionWeigth; // weight of the cohesion component in the blending
    float alignWeigth; // weight of the align component in the blending
};
#endif

Overwriting Flock.h


In [ ]:
%%writefile Flock.cpp
#include "Flock.h"

Flock::Flock(): boidsMap{}, neighborhoodMap{} {}

Flock::Flock(std::map<int, Boid>&& boidsMap): boidsMap{boidsMap} {
    
    updateNeighborhoodMap();
}

const std::map<int, std::vector<int>>& Flock::getNeighborhoodMap() const { return neighborhoodMap; }

const std::map<int, Boid>& Flock::getBoidsMap() const { return boidsMap; }

const std::vector<int>& Flock::getNeighborhood(const Boid& b) const { return neighborhoodMap.at(b.getId()); }

void Flock::setNeighborhoodDim(int value){ neighborhoodDim = value; }

void Flock::setBlendingWeigths(const int s, const int c, const int a){
    separationWeigth = s;
    cohesionWeigth = c;
    alignWeigth = a;
}

std::vector<int> Flock::computeNeighborhood(const Boid& b){
    
    std::vector<int> neighborhood{};
    for(const auto& elem : boidsMap){
        
        if(elem.first != b.getId() && vector3Distance(elem.second.getPosition(), b.getPosition()) <= neighborhoodDim){
            neighborhood.push_back(elem.first);
        }
    }

    return neighborhood;
}

void Flock::addBoid(const Boid& b){
    boidsMap.insert(std::pair<int,Boid>(b.getId(), b));
    neighborhoodMap.insert(std::pair<int,std::vector<int>>(b.getId(), std::vector<int>{}));
}

void Flock::updateNeighborhoodMap(){
    
    for(const auto& elem : boidsMap){
        if(neighborhoodMap.find(elem.first) != neighborhoodMap.end()){
            neighborhoodMap[elem.first] = computeNeighborhood(elem.second);
        }
        else{
            neighborhoodMap.insert(std::pair<int,std::vector<int>>(elem.first, computeNeighborhood(elem.second)));
        }
    }
}

void Flock::updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(auto& elem : boidsMap){
      
        getSeparationDirection(elem.first, separation);
        getCohesionDirection(elem.first, cohesion);
        getAlignDirection(elem.first, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            elem.second.setDirection(finalDirection);
        }

        elem.second.move(time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    updateNeighborhoodMap();
}

void Flock::getSeparationDirection(int b, float* separation) const{
    
    float* tmp = (float*) malloc(3*sizeof(float));
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sub(boidsMap.at(b).getPosition(), boidsMap.at(n).getPosition(), tmp);
        vector3Normalize(tmp);
        vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001), tmp);
        vector3Sum(separation, tmp, separation);
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void Flock::getCohesionDirection(int b, float* cohesion) const{
    
    float count = 0.0;
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(cohesion, boidsMap.at(n).getPosition(), cohesion);
        count++;
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, boidsMap.at(b).getPosition(), cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void Flock::getAlignDirection(int b, float* align) const{
    
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(align, boidsMap.at(n).getDirection(), align);
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void Flock::print() const{
     
    for(const auto& elem : boidsMap){
        elem.second.print();
    }
}

void Flock::printNeighborhoods() const{
     
    for(const auto& elem : boidsMap){
        std::cout << elem.first << ": ";
        for(const auto& elem : getNeighborhood(elem.second)){
            std::cout << elem << ", ";
        }
        std::cout << std::endl;
    }
}

Overwriting Flock.cpp


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const float*, const float*);

// returns the sum of the passed vectors
void vector3Sum(const float*, const float*, float*);

// returns the subtraction of the passed vectors
void vector3Sub(const float*, const float*, float*);

// returns the multiplication of the passed vectors
void vector3Mul(const float*, const float, float*);

// returns the blending of the passed vectors representing directions
void blendDirections(const float*, const float*, const float*, float*);

// returns the magnitude of the passed vector
float vector3Magnitude(const float*);

// normalizes the passed vector
void vector3Normalize(float*);

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const float* v, const float* w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

void vector3Sum(const float* v, const float* w, float* res){
    
    res[0] = v[0] + w[0];
    res[1] = v[1] + w[1];
    res[2] = v[2] + w[2];
}

void vector3Sub(const float* v, const float* w, float* res){
    
    res[0] = v[0] - w[0];
    res[1] = v[1] - w[1];
    res[2] = v[2] - w[2];
}

void vector3Mul(const float* v, const float n, float* res){
    
    res[0] = v[0] * n;
    res[1] = v[1] * n;
    res[2] = v[2] * n;
}

void blendDirections(const float* v, const float* w, const float* u, float* res){
    
    res[0] = 0;
    res[1] = 0;
    res[2] = 0;
    vector3Sum(res, v, res);
    vector3Sum(res, w, res);
    vector3Sum(res, u, res);
}

float vector3Magnitude(const float* v){
    return sqrt(pow(v[0], 2) + pow(v[1], 2) + pow(v[2], 2));
}

void vector3Normalize(float* v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [ ]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourSeq.cu Boid.cpp Flock.cpp Utils.cpp -o FlockingBehaviourSeq

# Execution
!./FlockingBehaviourSeq

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


CPU Flock generation...
    CPU elapsed time: 0.00161 (sec)
first: -17.36700
first: -1.93900
first: -33.01900
first: 0.77666
first: -0.27083
first: -0.56872
first: 7.90900
first: -27.28000
first: 21.53100
first: 0.68623
first: 0.60659
first: 0.40142
last: 0.68797
last: -0.14308
last: 0.71150
last: -18.72300
last: 49.92900
last: 0.34900
last: 0.72526
last: 0.43646
last: 0.53244
last: -19.47000
last: -22.25100
last: 6.98800


CPU neighborhoods generation...
    CPU elapsed time: 3.52921 (sec)


CPU Flock update...
    CPU elapsed time: 17.29297 (sec)

# Flocking Behaviour Parallel

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd "content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel"
%ls

/content/drive/MyDrive/GPU Computing/Progetto/FlockingBehaviourParallel
Boid.cpp  Flock.cpp  FlockingBehaviourPar     FlockingBehaviourPar.h  Utils.cpp
Boid.h    Flock.h    FlockingBehaviourPar.cu  src/                    Utils.h


In [2]:
%cd ..
%cd "FlockingBehaviourSequential"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourSequential'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [ ]:
%cd ..
%cd "FlockingBehaviourParallel"
%ls

/
[Errno 2] No such file or directory: 'FlockingBehaviourParallel'
/
bin/      dev/   lib32/  opt/         run/   tensorflow-1.15.2/  var/
boot/     etc/   lib64/  proc/        sbin/  tmp/
content/  home/  media/  python-apt/  srv/   tools/
datalab/  lib/   mnt/    root/        sys/   usr/


In [ ]:
%%writefile FlockingBehaviourPar.h
#include "Boid.h"
#include "Flock.h"
#include "Utils.h"
#include <stdlib.h>
#include <time.h> 
#include <iostream> 
#include <map> 
#include <vector> 
#include <curand_kernel.h>

typedef struct BoidData {
    float p1;
    float p2;
    float p3;
    float d1;
    float d2;
    float d3;
} BoidData;

void computeNeighborhoods();

void updateFlock(float);
void getSeparationDirection(int, float*);
void getCohesionDirection(int, float*);
void getAlignDirection(int, float*);
void moveBoid(int, float);

void printNeighborhoods();
void printFlock();
void printBoid(int);

Overwriting FlockingBehaviourPar.h


In [37]:
%%writefile FlockingBehaviourPar.cu
#include "FlockingBehaviourPar.h"

#define GEN_BLOCK_SIZE 128
#define GEN_GRID_SIZE 32
//384  128
//64   32
#define NEIGH_BLOCK_SIZE 256
//128
#define UPDATE_BLOCK_SIZE 256

float velocity = 20; // boid velocity in meters per second
double updateTime = 1.5; // update time of the simulation in seconds
float separationWeigth = 1; // weight of the separation component in the blending
float cohesionWeigth = 1; // weight of the cohesion component in the blending
float alignWeigth = 1; // weight of the align component in the blending
int flockDim = 10240; // 10000000 number of boids in the flock
float neighDim = 50; // 50 dimension of the neighborhood in meters
float tolerance = 0.1f; //tolerance for float comparison
int minRand = -50000; // -50000 minimum value that can be generated for initial position and direction
int maxRand = 50000; // 50000 maximum value that can be generated for initial position and direction
float decimals = 3; // 3 number of decimal digits in the generated values for initial position and direction
int iterations = 1; // number of updates
int generationsPerThread = 250; //2500 number of boids a thread must generate

__constant__ float velocityDev;
__constant__ double updateTimeDev;
__constant__ float separationWeigthDev;
__constant__ float cohesionWeigthDev;
__constant__ float alignWeigthDev;
__constant__ int flockDimDev;
__constant__ float neighDimDev;
__constant__ float toleranceDev;
__constant__ int minRandDev;
__constant__ int minMaxDiffDev;
__constant__ float divDev;
__constant__ int threadsNumDev;
__constant__ int generationsPerThreadDev;
__constant__ int lastThreadGenerationsDev;

float* flockData;
bool* neighborhoods;
bool* neighborhoodsSeq;

void computeNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				for(int j = 0; j < flockDim; j++){
						if(j > i){	
								neighborhoodsSeq[i*flockDim+j] = vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim) <= neighDim;
						}
						else if(i == j){
								neighborhoodsSeq[i*flockDim+j] = 0;
						}
						else{
								neighborhoodsSeq[i*flockDim+j] = neighborhoodsSeq[j*flockDim+i];
						}
				}
    }
}

void printBoid(int i){
		
		std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
		"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
}

void printFlock(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
				"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
    }
}

void printNeighborhoods(){
    
    for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoods[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }
}

void updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(int i = 0; i < flockDim; i++){
      
        getSeparationDirection(i, separation);
        getCohesionDirection(i, cohesion);
        getAlignDirection(i, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            flockData[i+3*flockDim] = finalDirection[0];
						flockData[i+4*flockDim] = finalDirection[1];
						flockData[i+5*flockDim] = finalDirection[2];
        }

        moveBoid(i, time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    computeNeighborhoods();
}

void getSeparationDirection(int i, float* separation){
    
    float* tmp = (float*) malloc(3*sizeof(float));
    for(int j = 0; j < flockDim; j++){
				if(neighborhoodsSeq[i*flockDim+j]){
						vector3Sub(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim, tmp);
						vector3Normalize(tmp);
						vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001), tmp);
						vector3Sum(separation, tmp, separation);
				}
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void getCohesionDirection(int i, float* cohesion){
    
    float count = 0.0;
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
						vector3Sum(cohesion, cohesion+1, cohesion+2, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim, cohesion);
						count++;
				}
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, cohesion+1, cohesion+2, flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void getAlignDirection(int i, float* align){
    
    for(int j = 0; j < flockDim; j++){
				if(neighborhoods[i*flockDim+j]){
        		vector3Sum(align, align+1, align+2, flockData+j+3*flockDim, flockData+j+4*flockDim, flockData+j+5*flockDim, align);
				}
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void moveBoid(int i, float time) { 
    
    for(int j = 0; j < 3; j++){
        flockData[i+j*flockDim] += flockData[i+j*flockDim] * velocity * time;
    }
}

/*
 * Initialize cuRAND states
 */
__global__ void initializeStates(uint seed, curandState* states) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		curand_init(seed, tid, 0, &states[tid]);
}

/*
 * Kernel for flock generation: generates boids with random initial position and direction
 */
__global__ void generateBoidsStatus(float* generated, curandState* states) {

	uint tid = threadIdx.x + blockDim.x * blockIdx.x;

	if(tid < threadsNumDev){
			
			curandState localState = states[tid];

			//to avoid accesses out of bounds
			//if the thread is the last make it generate the numbers for the remaining boids, otherwise make it generate generationsPerThread numbers
			int myGenerations = (tid == (threadsNumDev - 1)) * lastThreadGenerationsDev + !(tid == (threadsNumDev - 1)) * generationsPerThreadDev;
	
			float d1, d2, d3;
			float value;
			uint pos = tid - threadsNumDev;
			//myGenerations = myGenerations/2;
			for(uint i = 0; i < myGenerations; i++){
					
					d1 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					d2 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					d3 = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;

					value = sqrt(d1*d1 + d2*d2 + d3*d3);

					//if the magnitude is 0 avoid dividing for 0 and set the value to 0, otherwise calculate the value to normalize
					value = !(value == 0) * 1/(value + (value == 0));

					pos += (i <= lastThreadGenerationsDev) * threadsNumDev + (i > lastThreadGenerationsDev) * (threadsNumDev - 1);
					generated[pos] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					generated[pos + flockDimDev] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					generated[pos + 2 * flockDimDev] = (curand_uniform(&localState) * minMaxDiffDev + minRandDev) / divDev;
					generated[pos + 3 * flockDimDev] = d1 * value;
					generated[pos + 4 * flockDimDev] = d2 * value;
					generated[pos + 5 * flockDimDev] = d3 * value;
			}		
			
			states[tid] = localState;
	}
}

/*
 * Kernel for neighborhoods computation: computes distances and fills the boolean matrix
 */
__global__ void computeAllNeighborhoods(float* flockData, bool* neighborhoods) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		if(tid == 0){
				printf("aaaa: %.5f, %.5f", neighDimDev, toleranceDev);
		}

		if(tid < threadsNumDev){
				
				bool value;
				for(int i = 0; i < flockDimDev; i++){

						if(tid == i){
								neighborhoods[i*flockDimDev+tid] = 0;
						}
						else{
								//value = fabs(sqrt(pow(flockData[tid] - flockData[i], 2) + pow(flockData[tid+flockDimDev] - flockData[i+flockDimDev], 2) + pow(flockData[tid+2*flockDimDev] - flockData[i+2*flockDimDev], 2)) - neighDimDev) < toleranceDev;
								value = sqrt(pow(flockData[tid] - flockData[i], 2) + pow(flockData[tid+flockDimDev] - flockData[i+flockDimDev], 2) + pow(flockData[tid+2*flockDimDev] - flockData[i+2*flockDimDev], 2)) <= neighDimDev;
								neighborhoods[i*flockDimDev+tid] = value;
						}
				}
				
				/*
				uint i = (tid/flockDimDev) * flockDimDev/2;
				uint max = i + flockDimDev/2;
				tid = (tid>=flockDimDev) * (tid-flockDimDev) + (tid<flockDimDev) * tid;

				bool value;
				for(; i < max; i++){

						if(tid == i){
								neighborhoods[i*flockDimDev+tid] = 0;
						}
						else{
								value = sqrt(pow(flockData[tid] - flockData[i], 2) + pow(flockData[tid+flockDimDev] - flockData[i+flockDimDev], 2) + pow(flockData[tid+2*flockDimDev] - flockData[i+2*flockDimDev], 2)) <= neighDimDev;
								neighborhoods[i*flockDimDev+tid] = value;
						}
				}
				*/

				/*
				value = sqrt(pow(flockData[tid] - flockData[i], 2) + pow(flockData[tid+flockDimDev] - flockData[i+flockDimDev], 2) + pow(flockData[tid+2*flockDimDev] - flockData[i+2*flockDimDev], 2)) <= neighDimDev;
				//value = (tid == i) * 0 + !(tid == i) * value;
				neighborhoods[i*flockDimDev+tid] = value;
				//neighborhoods[tid*flockDimDev+i] = value;
				*/
		}
}

/*
 * Kernel for flock update: firstly determines the new direction of each boid based on its neighbors status and secondly its new position moving in the new direction at the given velocity 
 */
__global__ void updateFlock(float* flockData, bool* neighborhoods) {

		uint tid = threadIdx.x + blockDim.x * blockIdx.x;

		if(tid < threadsNumDev){
				
				
		}
}

int main(void) {

		device_name();

		// create events to measure time
		cudaEvent_t start, stop;
		cudaEventCreate(&start);
		cudaEventCreate(&stop);
		float milliseconds = 0;
		double cpuTimeStart;
		double cpuTime;

		// prepare for flock generation
		int minMaxDiff = maxRand - minRand;
		float div = pow(10.0, decimals);
		int numsToGenerate = flockDim * 6;

		float* flockData;
		curandState* states;
		int blockSize = GEN_BLOCK_SIZE;
		int gridSize = GEN_GRID_SIZE; 
		CHECK(cudaMallocManaged((void **) &flockData, numsToGenerate * sizeof(float)));
		CHECK(cudaMalloc((void **) &states, blockSize * gridSize * sizeof(curandState)));
		int threadsNum = (flockDim + generationsPerThread - 1) / generationsPerThread;
		int lastThreadGenerations = flockDim - generationsPerThread * (threadsNum - 1);

		// initialize all constant values
		cudaMemcpyToSymbol(velocityDev, &velocity, sizeof(velocityDev));
		cudaMemcpyToSymbol(updateTimeDev, &updateTime, sizeof(updateTimeDev));
		cudaMemcpyToSymbol(separationWeigthDev, &separationWeigth, sizeof(separationWeigthDev));
		cudaMemcpyToSymbol(cohesionWeigthDev, &cohesionWeigth, sizeof(cohesionWeigthDev));
		cudaMemcpyToSymbol(alignWeigthDev, &alignWeigth, sizeof(alignWeigthDev));
		cudaMemcpyToSymbol(flockDimDev, &flockDim, sizeof(flockDimDev));
		cudaMemcpyToSymbol(neighDimDev, &neighDim, sizeof(neighDimDev));
		cudaMemcpyToSymbol(toleranceDev, &tolerance, sizeof(toleranceDev));
		cudaMemcpyToSymbol(minRandDev, &minRand, sizeof(minRandDev));
		cudaMemcpyToSymbol(minMaxDiffDev, &minMaxDiff, sizeof(minMaxDiffDev));
		cudaMemcpyToSymbol(divDev, &div, sizeof(divDev));
		cudaMemcpyToSymbol(threadsNumDev, &threadsNum, sizeof(threadsNumDev));
		cudaMemcpyToSymbol(generationsPerThreadDev, &generationsPerThread, sizeof(generationsPerThreadDev));
		cudaMemcpyToSymbol(lastThreadGenerationsDev, &lastThreadGenerations, sizeof(lastThreadGenerationsDev));

		printf("\nthreadsNum: %i\n", threadsNum);
		printf("effthreadsNum: %i\n", blockSize * gridSize);
		printf("genperthread: %i\n", generationsPerThread);
		printf("lastthreadgen: %i\n", lastThreadGenerations);
		if(threadsNum > blockSize * gridSize){
				std::cout << "\nNot enough threads" << std::endl;
		}

		// generate flock
		printf("\n\nGPU Flock generation...\n");
		cudaEventRecord(start);

		initializeStates<<<gridSize, blockSize>>>(time(NULL), states);

		generateBoidsStatus<<<gridSize, blockSize>>>(flockData, states);

		cudaEventRecord(stop);
		CHECK(cudaEventSynchronize(stop));
		cudaEventElapsedTime(&milliseconds, start, stop);
		printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

		//print the first 10 and the last 10 boids to check the generation correctness 
		for(int i = 0; i < 10; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
					"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
				printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+i+flockDim*3, flockData+i+flockDim*4, flockData+i+flockDim*5));
		}

		for(int i = 0; i < 10; i++){
				std::cout << std::setprecision(4) << "Boid " << flockDim-1-i << ": " << "pos(" << flockData[flockDim-1-i] << ", " << flockData[flockDim-1-i+flockDim] << ", " << flockData[flockDim-1-i+flockDim*2] << 
					"); dir(" << flockData[flockDim-1-i+flockDim*3] << ", " << flockData[flockDim-1-i+flockDim*4] << ", " << flockData[flockDim-1-i+flockDim*5] << ")" << std::endl;
				printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+flockDim-1-i+flockDim*3, flockData+flockDim-1-i+flockDim*4, flockData+flockDim-1-i+flockDim*5));
		}

		for(int i = flockDim/2-5; i < flockDim/2+5; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
					"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
				printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+i+flockDim*3, flockData+i+flockDim*4, flockData+i+flockDim*5));
		}

		// generate flock sequentially to measure the speed-up
		/*
		float* flockDataSeq = (float*) malloc(numsToGenerate * sizeof(float));

		printf("\n\nCPU Flock generation...\n");
		cpuTimeStart = seconds();

		for(int i = 0; i < numsToGenerate; i+=6){
			flockDataSeq[i] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+1] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+2] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+3] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+4] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
			flockDataSeq[i+5] = (minRand + rand() % (maxRand + 1 - minRand)) / div;
		
			vector3Normalize(flockDataSeq+i+3);
		}

		cpuTime = seconds() - cpuTimeStart;
		printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

		printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));

		free(flockDataSeq);
		*/

		CHECK(cudaFree(states));

		// prepare for neighborhood calculation
		// neighborhoods data stored in a boolean matrix
		CHECK(cudaMallocManaged((void **) &neighborhoods, flockDim * flockDim * sizeof(bool)));
		threadsNum = flockDim; //flockDim*2; 
		int neighBlockSize = NEIGH_BLOCK_SIZE;
		int neighGridSize = (threadsNum + blockSize - 1)/blockSize;

		printf("\nthreadsNum: %i\n", threadsNum);
		printf("gridSize: %i\n", neighGridSize);
		printf("effthreadsNum: %i\n", neighBlockSize * neighGridSize);
		if(threadsNum > neighBlockSize * neighGridSize){
				std::cout << "\nNot enough threads" << std::endl;
		}

		// update total threads number in constant memory
		cudaMemcpyToSymbol(threadsNumDev, &threadsNum, sizeof(threadsNumDev));

		// compute all neighborhoods
		printf("\n\nGPU Neighborhoods computation...\n");
		cudaEventRecord(start);

		computeAllNeighborhoods<<<neighGridSize, neighBlockSize>>>(flockData, neighborhoods);

		cudaEventRecord(stop);
		CHECK(cudaEventSynchronize(stop));
		cudaEventElapsedTime(&milliseconds, start, stop);
		printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);

		for(int i = 0; i < 12; i++){
				printf("first neigh: %d\n", neighborhoods[i]);
		}
		for(int i = 0; i < 12; i++){
				printf("last neigh: %d\n", neighborhoods[(flockDim-1)*flockDim+flockDim-1-i]);
		}

		/*
		for(int i = 0; i < flockDim; i++){
				std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
				"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
    }
		*/

		//printFlock();		
		//std::cout << std::endl;
		//printNeighborhoods();
		//std::cout << std::endl;
		//std::cout << std::endl;

		// calculate neighborhoods sequentially to check the result and measure the speed-up
		
		neighborhoodsSeq = (bool*) malloc(flockDim * flockDim * sizeof(bool));

		printf("\n\nCPU Neighborhoods computation...\n");
		cpuTimeStart = seconds();

		float dist;
		for(int i = 0; i < flockDim; i++){
				for(int j = 0; j < flockDim; j++){
						if(j > i){	
								dist = vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim);
								neighborhoodsSeq[i*flockDim+j] = (dist <= neighDim);
						}
						else if(i == j){
								neighborhoodsSeq[i*flockDim+j] = 0;
						}
						else{
								neighborhoodsSeq[i*flockDim+j] = neighborhoodsSeq[j*flockDim+i];
						}
				}
    }

		cpuTime = seconds() - cpuTimeStart;
		printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);

		printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));
		

		
		bool correct = 1;
		for(int i = 0; i < flockDim; i++){
				
				if(!correct){
						break;
				}

				for(int j = 0; j < flockDim; j++){
						if(neighborhoodsSeq[i*flockDim+j] != neighborhoods[i*flockDim+j] && vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim) != neighDim){
								correct = 0;
								printf("seq: %i, %.5f --- par: %i, %.5f\n", neighborhoodsSeq[i*flockDim+j], vector3Distance(flockData+i, flockData+i+flockDim, flockData+i+2*flockDim, flockData+j, flockData+j+flockDim, flockData+j+2*flockDim),
								        neighborhoods[i*flockDim+j], sqrt(pow(flockData[i] - flockData[j], 2) + pow(flockData[i+flockDim] - flockData[j+flockDim], 2) + pow(flockData[i+2*flockDim] - flockData[j+2*flockDim], 2)));
								printf("i: %i --- j: %i\n", i,j);
								
								std::cout << std::setprecision(4) << "Boid " << i << ": " << "pos(" << flockData[i] << ", " << flockData[i+flockDim] << ", " << flockData[i+flockDim*2] << 
									"); dir(" << flockData[i+flockDim*3] << ", " << flockData[i+flockDim*4] << ", " << flockData[i+flockDim*5] << ")" << std::endl;
								printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+i+flockDim*3, flockData+i+flockDim*4, flockData+i+flockDim*5));

								std::cout << std::setprecision(4) << "Boid " << j << ": " << "pos(" << flockData[j] << ", " << flockData[j+flockDim] << ", " << flockData[j+flockDim*2] << 
									"); dir(" << flockData[j+flockDim*3] << ", " << flockData[j+flockDim*4] << ", " << flockData[j+flockDim*5] << ")" << std::endl;
								printf("Boid direction magnitude: %.5f\n", vector3Magnitude(flockData+j+flockDim*3, flockData+j+flockDim*4, flockData+j+flockDim*5));
								break;
						}
				}
    }

		printf("\nNeighborhoods computation correctness: %i\n\n", correct);
/*
		for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoods[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }

		for(int i = 0; i < flockDim; i++){
				std::cout << i << ": ";
				for(int j = 0; j < flockDim; j++){
						if(neighborhoodsSeq[i*flockDim+j]){
								std::cout << j << ", ";
						}
				}
				std::cout << std::endl;
    }
*/
		//prepare for flock updates
		threadsNum = flockDim*2;
		blockSize = UPDATE_BLOCK_SIZE;
		gridSize = (threadsNum + blockSize - 1)/blockSize;

		// start simulation loop that updates the flock each updateTime
		double loopStart = seconds();
		double tmpTime = updateTime;
		while(iterations > 0){

				auto duration = seconds() - loopStart;
				if(duration >= tmpTime)
				{
						// update the flock
						printf("\n\nGPU Flock update...\n");
						cudaEventRecord(start);

						updateFlock<<<gridSize, blockSize>>>(flockData, neighborhoods);

						cudaEventRecord(stop);
						CHECK(cudaEventSynchronize(stop));
						cudaEventElapsedTime(&milliseconds, start, stop);
						printf("    GPU elapsed time: %.5f (sec)\n", milliseconds / 1000);
					
						// update the flock sequentially to check the result and measure the speed-up
						

						printf("\n\nCPU Flock update...\n");
						double cpuTimeStart = seconds();

						updateFlock(updateTime);
					
						double cpuTime = seconds() - cpuTimeStart;
						printf("    CPU elapsed time: %.5f (sec)\n", cpuTime);
					
						printf("				Speedup: %.2f\n", cpuTime/(milliseconds / 1000));
						
						//-------
						
					
						computeAllNeighborhoods<<<neighGridSize, neighBlockSize>>>(flockData, neighborhoods);
					
						tmpTime += updateTime;
						iterations--;

						//std::cout << std::endl;
						//printFlock();
					
						//std::cout << std::endl;
						//printNeighborhoods();
				}
		}

		CHECK(cudaFree(flockData));
		CHECK(cudaFree(neighborhoods));
		//free(neighborhoodsSeq);
		
		cudaDeviceReset();
		return 0;
}

Overwriting FlockingBehaviourPar.cu


In [ ]:
%%writefile Boid.h
#include <vector>
#include <iostream>
#include <iomanip>
#include <string>

#ifndef BOID_H
#define BOID_H
class Boid{
    
    public:
    Boid();

    Boid(const int, const float, float*, float*);

    float* getDirection() const;
    float* getPosition() const;
    const int& getId() const;
    void setDirection(float*);
    void setPosition(float*);

    // move the boid by updating its position based on its current direction, velocity and time passed since the last update
    void move(float);

    void print() const;

    private:
    int id; // unique id of the boid
    float velocity; // boid velocity in meters per second
    float* position; // one unit is one meter
    float* direction; // normalized vector
};
#endif

Overwriting Boid.h


In [ ]:
%%writefile Boid.cpp
#include "Boid.h"

Boid::Boid(const int id, const float vel, float* pos, float* dir): id{id}, velocity{vel}, direction{dir}, position{pos} {}

float* Boid::getDirection() const { return direction; }

float* Boid::getPosition() const { return position; }

const int& Boid::getId() const { return id; }

void Boid::setDirection(float* dir) { 
    direction[0] = dir[0]; 
    direction[1] = dir[1]; 
    direction[2] = dir[2]; 
}

void Boid::setPosition(float* pos) { 
    position[0] = pos[0];
    position[1] = pos[1];
    position[2] = pos[2];
}

void Boid::move(float time) { 
    
    for(int i = 0; i < 3; i++){
        position[i] += direction[i] * velocity * time;
    }
}

void Boid::print() const{
    std::cout << std::setprecision(4) << "Boid " << id << ": " << "pos(" << position[0] << ", " << position[1] << ", " << position[2] << 
    "); dir(" << direction[0] << ", " << direction[1] << ", " << direction[2] << ")" << std::endl;
}

Overwriting Boid.cpp


In [ ]:
%%writefile Flock.h
#include <iostream>
#include <map>
#include <vector>
#include "Boid.h"
#include "Utils.h"

#ifndef FLOCK_H
#define FLOCK_H
class Flock{
    
    public:
    Flock();
    Flock(std::map<int, Boid>&&);

    const std::map<int, std::vector<int>>& getNeighborhoodMap() const;
    const std::map<int, Boid>& getBoidsMap() const;

    const std::vector<int>& getNeighborhood(const Boid&) const;
    void setNeighborhoodDim(const int);
    void setBlendingWeigths(const int, const int, const int);

    // returns the list of boids that are nearer than neighborhoodDim from the passed boid
    std::vector<int> computeNeighborhood(const Boid&);

    // adds a boid to the flock
    void addBoid(const Boid&);

    // computes the neighborhoods of each boid based on the current situation and puts them inside the neighborhoodMap
    void updateNeighborhoodMap();

    // for each boid computes all the components that influence its direction, blends them and set the resulting 
    // direction as the current direction of the boid only if it is not (0,0,0). otherwise the old direction is maintained
    void updateFlock(float);

    void getSeparationDirection(const int, float*) const;
    void getCohesionDirection(const int, float*) const;
    void getAlignDirection(const int, float*) const;

    void print() const;
    void printNeighborhoods() const;

    private:
    std::map<int, std::vector<int>> neighborhoodMap; // associates the neighborhood of a boid to the id of the boid
    std::map<int, Boid> boidsMap; // associates a boid to its id
    float neighborhoodDim; // dimension of the neighborhood in meters
    float separationWeigth; // weight of the separation component in the blending
    float cohesionWeigth; // weight of the cohesion component in the blending
    float alignWeigth; // weight of the align component in the blending
};
#endif

Overwriting Flock.h


In [ ]:
%%writefile Flock.cpp
#include "Flock.h"

Flock::Flock(): boidsMap{}, neighborhoodMap{} {}

Flock::Flock(std::map<int, Boid>&& boidsMap): boidsMap{boidsMap} {
    
    updateNeighborhoodMap();
}

const std::map<int, std::vector<int>>& Flock::getNeighborhoodMap() const { return neighborhoodMap; }

const std::map<int, Boid>& Flock::getBoidsMap() const { return boidsMap; }

const std::vector<int>& Flock::getNeighborhood(const Boid& b) const { return neighborhoodMap.at(b.getId()); }

void Flock::setNeighborhoodDim(int value){ neighborhoodDim = value; }

void Flock::setBlendingWeigths(const int s, const int c, const int a){
    separationWeigth = s;
    cohesionWeigth = c;
    alignWeigth = a;
}

std::vector<int> Flock::computeNeighborhood(const Boid& b){
    
    std::vector<int> neighborhood{};
    for(const auto& elem : boidsMap){
        
        if(elem.first != b.getId() && vector3Distance(elem.second.getPosition(), b.getPosition()) <= neighborhoodDim){
            neighborhood.push_back(elem.first);
        }
    }

    return neighborhood;
}

void Flock::addBoid(const Boid& b){
    boidsMap.insert(std::pair<int,Boid>(b.getId(), b));
    neighborhoodMap.insert(std::pair<int,std::vector<int>>(b.getId(), std::vector<int>{}));
}

void Flock::updateNeighborhoodMap(){
    
    for(const auto& elem : boidsMap){
        if(neighborhoodMap.find(elem.first) != neighborhoodMap.end()){
            neighborhoodMap[elem.first] = computeNeighborhood(elem.second);
        }
        else{
            neighborhoodMap.insert(std::pair<int,std::vector<int>>(elem.first, computeNeighborhood(elem.second)));
        }
    }
}

void Flock::updateFlock(float time){
    
    float* cohesion = (float*) malloc(3*sizeof(float));
    float* separation = (float*) malloc(3*sizeof(float));
    float* align = (float*) malloc(3*sizeof(float));
    float* finalDirection = (float*) malloc(3*sizeof(float));
		for(auto& elem : boidsMap){
      
        getSeparationDirection(elem.first, separation);
        getCohesionDirection(elem.first, cohesion);
        getAlignDirection(elem.first, align);

        blendDirections(separation, cohesion, align, finalDirection);
        if (finalDirection[0] != 0 || finalDirection[1] != 0 || finalDirection[2] != 0) {
            elem.second.setDirection(finalDirection);
        }

        elem.second.move(time);
    }

    free(cohesion);
    free(separation);
    free(align);
    free(finalDirection);

    updateNeighborhoodMap();
}

void Flock::getSeparationDirection(int b, float* separation) const{
    
    float* tmp = (float*) malloc(3*sizeof(float));
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sub(boidsMap.at(b).getPosition(), boidsMap.at(n).getPosition(), tmp);
        vector3Normalize(tmp);
        vector3Mul(tmp, 1/(vector3Magnitude(tmp) + 0.0001), tmp);
        vector3Sum(separation, tmp, separation);
    }

    vector3Normalize(separation);
    vector3Mul(separation, separationWeigth, separation);

    free(tmp);
}

void Flock::getCohesionDirection(int b, float* cohesion) const{
    
    float count = 0.0;
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(cohesion, boidsMap.at(n).getPosition(), cohesion);
        count++;
    }

    if(count != 0){
        vector3Mul(cohesion, 1.0/count, cohesion);
        vector3Sub(cohesion, boidsMap.at(b).getPosition(), cohesion);
    }

    vector3Normalize(cohesion);
    vector3Mul(cohesion, cohesionWeigth, cohesion);
}

void Flock::getAlignDirection(int b, float* align) const{
    
    for(const auto& n : neighborhoodMap.at(b)){
        vector3Sum(align, boidsMap.at(n).getDirection(), align);
    }

    vector3Normalize(align);
    vector3Mul(align, alignWeigth, align);
}

void Flock::print() const{
     
    for(const auto& elem : boidsMap){
        elem.second.print();
    }
}

void Flock::printNeighborhoods() const{
     
    for(const auto& elem : boidsMap){
        std::cout << elem.first << ": ";
        for(const auto& elem : getNeighborhood(elem.second)){
            std::cout << elem << ", ";
        }
        std::cout << std::endl;
    }
}

Overwriting Flock.cpp


In [ ]:
%%writefile Utils.h
#include <vector>
#include <math.h>
#include <chrono>
#include <sys/time.h>
#include <cuda_runtime_api.h>
#include <cuda.h>
#include <stdio.h>

#ifndef UTILS_H
#define UTILS_H

typedef unsigned long ulong;
typedef unsigned int uint;

// returns the distance between the passed vectors
float vector3Distance(const float*, const float*);
float vector3Distance(const float*, const float*, const float*, const float*, const float*, const float*);

// returns the sum of the passed vectors
void vector3Sum(const float*, const float*, float*);
void vector3Sum(const float*, const float*, const float*, const float*, const float*, const float*, float*);

// returns the subtraction of the passed vectors
void vector3Sub(const float*, const float*, float*);
void vector3Sub(const float*, const float*, const float*, const float*, const float*, const float*, float*);

// returns the multiplication of the passed vectors
void vector3Mul(const float*, const float, float*);
void vector3Mul(const float*, const float*, const float*, const float, float*);

// returns the blending of the passed vectors representing directions
void blendDirections(const float*, const float*, const float*, float*);

// returns the magnitude of the passed vector
float vector3Magnitude(const float*);
float vector3Magnitude(const float*, const float*, const float*);

// normalizes the passed vector
void vector3Normalize(float*);
void vector3Normalize(float*, float*, float*);

#define CHECK(call)                                                           \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
    }                                                                          \
}

#define CHECK_CUBLAS(call)                                                     \
{                                                                              \
    cublasStatus_t err;                                                        \
    if ((err = (call)) != CUBLAS_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CUBLAS error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CURAND(call)                                                     \
{                                                                              \
    curandStatus_t err;                                                        \
    if ((err = (call)) != CURAND_STATUS_SUCCESS)                               \
    {                                                                          \
        fprintf(stderr, "Got CURAND error %d at %s:%d\n", err, __FILE__,       \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUFFT(call)                                                      \
{                                                                              \
    cufftResult err;                                                           \
    if ( (err = (call)) != CUFFT_SUCCESS)                                      \
    {                                                                          \
        fprintf(stderr, "Got CUFFT error %d at %s:%d\n", err, __FILE__,        \
                __LINE__);                                                     \
        exit(1);                                                               \
    }                                                                          \
}

#define CHECK_CUSPARSE(call)                                                   \
{                                                                              \
    cusparseStatus_t err;                                                      \
    if ((err = (call)) != CUSPARSE_STATUS_SUCCESS)                             \
    {                                                                          \
        fprintf(stderr, "Got error %d at %s:%d\n", err, __FILE__, __LINE__);   \
        cudaError_t cuda_err = cudaGetLastError();                             \
        if (cuda_err != cudaSuccess)                                           \
        {                                                                      \
            fprintf(stderr, "  CUDA error \"%s\" also detected\n",             \
                    cudaGetErrorString(cuda_err));                             \
        }                                                                      \
        exit(1);                                                               \
    }                                                                          \
}

inline double seconds() {
    struct timeval tp;
    struct timezone tzp;
    int i = gettimeofday(&tp, &tzp);
    return ((double)tp.tv_sec + (double)tp.tv_usec * 1.e-6);
}

inline void device_name() {
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));
}
#endif

Overwriting Utils.h


In [ ]:
%%writefile Utils.cpp
#include "Utils.h"

float vector3Distance(const float* v, const float* w){
    return sqrt(pow(w[0] - v[0], 2) + pow(w[1] - v[1], 2) + pow(w[2] - v[2], 2));
}

float vector3Distance(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z){
    return sqrt(pow(a[0] - x[0], 2) + pow(b[0] - y[0], 2) + pow(c[0] - z[0], 2));
}

void vector3Sum(const float* v, const float* w, float* res){
    
    res[0] = v[0] + w[0];
    res[1] = v[1] + w[1];
    res[2] = v[2] + w[2];
}

void vector3Sum(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z, float* res){
    
    res[0] = a[0] + x[0];
    res[1] = b[0] + y[0];
    res[2] = c[0] + z[0];
}

void vector3Sub(const float* v, const float* w, float* res){
    
    res[0] = v[0] - w[0];
    res[1] = v[1] - w[1];
    res[2] = v[2] - w[2];
}

void vector3Sub(const float* a, const float* b, const float* c, const float* x, const float* y, const float* z, float* res){
    
    res[0] = a[0] - x[0];
    res[1] = b[0] - y[0];
    res[2] = c[0] - z[0];
}

void vector3Mul(const float* v, const float n, float* res){
    
    res[0] = v[0] * n;
    res[1] = v[1] * n;
    res[2] = v[2] * n;
}

void vector3Mul(const float* a, const float* b, const float* c, const float n, float* res){
    
    res[0] = a[0] * n;
    res[1] = b[0] * n;
    res[2] = c[0] * n;
}

void blendDirections(const float* v, const float* w, const float* u, float* res){
    
    res[0] = 0;
    res[1] = 0;
    res[2] = 0;
    vector3Sum(res, v, res);
    vector3Sum(res, w, res);
    vector3Sum(res, u, res);
}

float vector3Magnitude(const float* v){
    return sqrt(pow(v[0], 2) + pow(v[1], 2) + pow(v[2], 2));
}

float vector3Magnitude(const float* a, const float* b, const float* c){
    return sqrt(pow(a[0], 2) + pow(b[0], 2) + pow(c[0], 2));
}

void vector3Normalize(float* v){
    if(v[0] != 0 || v[1] != 0 || v[2] != 0){
        float magnitude = vector3Magnitude(v);
        v[0] *= (1/magnitude);
        v[1] *= (1/magnitude);
        v[2] *= (1/magnitude);
    }
}

void vector3Normalize(float* a, float* b, float* c){
    if(a[0] != 0 || b[0] != 0 || c[0] != 0){
        float magnitude = vector3Magnitude(a,b,c);
        a[0] *= (1/magnitude);
        b[0] *= (1/magnitude);
        c[0] *= (1/magnitude);
    }
}

Overwriting Utils.cpp


In [39]:
# Compilation (e.g. -arch=sm_37 if compute capability is 3.7)
!nvcc -arch=sm_37 FlockingBehaviourPar.cu Boid.cpp Flock.cpp Utils.cpp -lcurand -o FlockingBehaviourPar

# Execution
!./FlockingBehaviourPar

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
device 0: Tesla K80

threadsNum: 41
effthreadsNum: 4096
genperthread: 250
lastthreadgen: 240


GPU Flock generation...
    GPU elapsed time: 0.00853 (sec)
Boid 0: pos(-40.46, 17.46, 13.57); dir(-0.6996, -0.05495, 0.7125)
Boid direction magnitude: 1.00000
Boid 1: pos(30.31, 39.21, 26.35); dir(0.4154, 0.662, 0.6238)
Boid direction magnitude: 1.00000
Boid 2: pos(-44.35, -0.5467, -16.2); dir(0.0445, 0.6814, -0.7306)
Boid direction magnitude: 1.00000
Boid 3: pos(-11.56, 13.82, 9.007); dir(0.1639, 0.1897, -0.9681)
Boid direction magnitude: 1.00000
Boid 4: pos(-34.97, 24.78, 5.589); dir(0.7559, 0.1976, 0.6241)
Boid direction magnitude: 1.00000
Boid 5: pos(5.942, -41.15, -34.33); dir(-0.3254, 0.09767, -0.9405)
Boid direction magnitude: 1.00000
Boid 6: pos(-13.84, 1.825, -1.037); dir(